Evaluate the performance of different types of optimizer on a LeNet-5 network using MNIST data. At least you need to evaluate SGD, AdaGrad, RMSprop. 

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

In [2]:
import numpy as np
from datetime import datetime 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms

import matplotlib.pyplot as plt

# check device
#DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# Preparing for Data
print('==> Preparing data..')

"""
# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

"""

==> Preparing data..


"\n# Training Data augmentation\ntransform_train = transforms.Compose([\n    transforms.RandomCrop(32, padding=4),\n    transforms.RandomHorizontalFlip(),\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n# Testing Data preparation\ntransform_test = transforms.Compose([\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n\n#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')\n\n"

In [4]:
# Preparing for Data
print('==> Preparing data..')

# define transforms
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor()])


==> Preparing data..


In [5]:
#Defining the convolutional neural network
class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = torch.flatten(out, 1) # flatten all dimensions except batch
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
 
        return out

In [6]:
model1 = LeNet()

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model1.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model1):,} trainable parameters')

The model has 61,706 trainable parameters


In [8]:
from torchvision import models
from torchsummary import summary

In [9]:
print(model1)

summary(model1)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            156
├─MaxPool2d: 1-2                         --
├─Conv2d: 1-3                            2,416
├─Linear: 1-4                            48,120
├─Linear: 1-5                            10,164
├─Linear: 1-6                            850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            156
├─MaxPool2d: 1-2                         --
├─Conv2d: 1-3                            2,416
├─Linear: 1-4                            48,120
├─Linear: 1-5                            10,164
├─Linear: 1-6                            850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0

In [10]:
########################################################################
# 3. Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Let's use a Classification Cross-Entropy loss and SGD with momentum.

#import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [11]:
def train(model, device, train_loader, optimizer, epoch):
    
    model.train()
    count = 0
    train_loss = 0
    total = 0 
    correct = 0
    
    total_step = len(train_loader)
    
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = data.to(device), target.to(device)
        

        #forward pass
        y_pred = model(data)

        loss = criterion(y_pred, target)

        acc = calculate_accuracy(y_pred, target)
        
        
        #Backward pass
        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()
        
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            

In [12]:
def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [14]:
def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 50
    lr = 0.05
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.MNIST(root='mnist_data', train=True, download=True, transform=transforms)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.MNIST(root='mnist_data', train=False, download=True, transform=transforms)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting mnist_data\MNIST\raw\train-images-idx3-ubyte.gz to mnist_data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting mnist_data\MNIST\raw\train-labels-idx1-ubyte.gz to mnist_data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Train Epoch: 34 [46080/60000 (77%)]	Loss: 0.010579
Train Epoch: 34 [47360/60000 (79%)]	Loss: 0.025055
Train Epoch: 34 [48640/60000 (81%)]	Loss: 0.008830
Train Epoch: 34 [49920/60000 (83%)]	Loss: 0.020727
Train Epoch: 34 [51200/60000 (85%)]	Loss: 0.006568
Train Epoch: 34 [52480/60000 (87%)]	Loss: 0.004445
Train Epoch: 34 [53760/60000 (90%)]	Loss: 0.052917
Train Epoch: 34 [55040/60000 (92%)]	Loss: 0.025083
Train Epoch: 34 [56320/60000 (94%)]	Loss: 0.027316
Train Epoch: 34 [57600/60000 (96%)]	Loss: 0.009748
Train Epoch: 34 [58880/60000 (98%)]	Loss: 0.022785

Test set: Average loss: -13.5627, Accuracy: 9861/10000 (99%)

Train Epoch: 35 [0/60000 (0%)]	Loss: 0.049488
Train Epoch: 35 [1280/60000 (2%)]	Loss: 0.007471
Train Epoch: 35 [2560/60000 (4%)]	Loss: 0.003070
Train Epoch: 35 [3840/60000 (6%)]	Loss: 0.067377
Train Epoch: 35 [5120/60000 (9%)]	Loss: 0.004611
Train Epoch: 35 [6400/60000 (11%)]	Loss: 0.010191
Train Epoch: 35 [7680/60000 (13%)]	Loss: 0.027036
Train Epoch: 35 [8960/60000 (15%)]

Train Epoch: 38 [6400/60000 (11%)]	Loss: 0.009800
Train Epoch: 38 [7680/60000 (13%)]	Loss: 0.008639
Train Epoch: 38 [8960/60000 (15%)]	Loss: 0.016925
Train Epoch: 38 [10240/60000 (17%)]	Loss: 0.020581
Train Epoch: 38 [11520/60000 (19%)]	Loss: 0.003723
Train Epoch: 38 [12800/60000 (21%)]	Loss: 0.008554
Train Epoch: 38 [14080/60000 (23%)]	Loss: 0.016190
Train Epoch: 38 [15360/60000 (26%)]	Loss: 0.011915
Train Epoch: 38 [16640/60000 (28%)]	Loss: 0.001212
Train Epoch: 38 [17920/60000 (30%)]	Loss: 0.005239
Train Epoch: 38 [19200/60000 (32%)]	Loss: 0.021137
Train Epoch: 38 [20480/60000 (34%)]	Loss: 0.006744
Train Epoch: 38 [21760/60000 (36%)]	Loss: 0.024128
Train Epoch: 38 [23040/60000 (38%)]	Loss: 0.032338
Train Epoch: 38 [24320/60000 (41%)]	Loss: 0.006777
Train Epoch: 38 [25600/60000 (43%)]	Loss: 0.002173
Train Epoch: 38 [26880/60000 (45%)]	Loss: 0.012823
Train Epoch: 38 [28160/60000 (47%)]	Loss: 0.011700
Train Epoch: 38 [29440/60000 (49%)]	Loss: 0.002161
Train Epoch: 38 [30720/60000 (51%)

Train Epoch: 41 [28160/60000 (47%)]	Loss: 0.002236
Train Epoch: 41 [29440/60000 (49%)]	Loss: 0.005734
Train Epoch: 41 [30720/60000 (51%)]	Loss: 0.027557
Train Epoch: 41 [32000/60000 (53%)]	Loss: 0.000428
Train Epoch: 41 [33280/60000 (55%)]	Loss: 0.017320
Train Epoch: 41 [34560/60000 (58%)]	Loss: 0.043025
Train Epoch: 41 [35840/60000 (60%)]	Loss: 0.017415
Train Epoch: 41 [37120/60000 (62%)]	Loss: 0.013971
Train Epoch: 41 [38400/60000 (64%)]	Loss: 0.030597
Train Epoch: 41 [39680/60000 (66%)]	Loss: 0.009533
Train Epoch: 41 [40960/60000 (68%)]	Loss: 0.007979
Train Epoch: 41 [42240/60000 (70%)]	Loss: 0.030478
Train Epoch: 41 [43520/60000 (72%)]	Loss: 0.001835
Train Epoch: 41 [44800/60000 (75%)]	Loss: 0.039632
Train Epoch: 41 [46080/60000 (77%)]	Loss: 0.005109
Train Epoch: 41 [47360/60000 (79%)]	Loss: 0.008314
Train Epoch: 41 [48640/60000 (81%)]	Loss: 0.016262
Train Epoch: 41 [49920/60000 (83%)]	Loss: 0.007715
Train Epoch: 41 [51200/60000 (85%)]	Loss: 0.047491
Train Epoch: 41 [52480/60000 (8

Train Epoch: 44 [49920/60000 (83%)]	Loss: 0.018632
Train Epoch: 44 [51200/60000 (85%)]	Loss: 0.098088
Train Epoch: 44 [52480/60000 (87%)]	Loss: 0.083026
Train Epoch: 44 [53760/60000 (90%)]	Loss: 0.030078
Train Epoch: 44 [55040/60000 (92%)]	Loss: 0.011457
Train Epoch: 44 [56320/60000 (94%)]	Loss: 0.037959
Train Epoch: 44 [57600/60000 (96%)]	Loss: 0.005354
Train Epoch: 44 [58880/60000 (98%)]	Loss: 0.091494

Test set: Average loss: -13.4196, Accuracy: 9833/10000 (98%)

Train Epoch: 45 [0/60000 (0%)]	Loss: 0.009945
Train Epoch: 45 [1280/60000 (2%)]	Loss: 0.048143
Train Epoch: 45 [2560/60000 (4%)]	Loss: 0.032423
Train Epoch: 45 [3840/60000 (6%)]	Loss: 0.014257
Train Epoch: 45 [5120/60000 (9%)]	Loss: 0.043665
Train Epoch: 45 [6400/60000 (11%)]	Loss: 0.017106
Train Epoch: 45 [7680/60000 (13%)]	Loss: 0.012416
Train Epoch: 45 [8960/60000 (15%)]	Loss: 0.018879
Train Epoch: 45 [10240/60000 (17%)]	Loss: 0.001163
Train Epoch: 45 [11520/60000 (19%)]	Loss: 0.014956
Train Epoch: 45 [12800/60000 (21%)]

Train Epoch: 48 [10240/60000 (17%)]	Loss: 0.017852
Train Epoch: 48 [11520/60000 (19%)]	Loss: 0.015559
Train Epoch: 48 [12800/60000 (21%)]	Loss: 0.006543
Train Epoch: 48 [14080/60000 (23%)]	Loss: 0.017528
Train Epoch: 48 [15360/60000 (26%)]	Loss: 0.034323
Train Epoch: 48 [16640/60000 (28%)]	Loss: 0.023270
Train Epoch: 48 [17920/60000 (30%)]	Loss: 0.011350
Train Epoch: 48 [19200/60000 (32%)]	Loss: 0.000826
Train Epoch: 48 [20480/60000 (34%)]	Loss: 0.062428
Train Epoch: 48 [21760/60000 (36%)]	Loss: 0.010705
Train Epoch: 48 [23040/60000 (38%)]	Loss: 0.013548
Train Epoch: 48 [24320/60000 (41%)]	Loss: 0.008069
Train Epoch: 48 [25600/60000 (43%)]	Loss: 0.046292
Train Epoch: 48 [26880/60000 (45%)]	Loss: 0.009511
Train Epoch: 48 [28160/60000 (47%)]	Loss: 0.034482
Train Epoch: 48 [29440/60000 (49%)]	Loss: 0.016702
Train Epoch: 48 [30720/60000 (51%)]	Loss: 0.032748
Train Epoch: 48 [32000/60000 (53%)]	Loss: 0.003509
Train Epoch: 48 [33280/60000 (55%)]	Loss: 0.012984
Train Epoch: 48 [34560/60000 (5

In [ ]:
#%config InlineBackend.figure_format = 'retina'